In [ ]:
from math import factorial, exp
import matplotlib.pyplot as plt
import numpy as np

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def erlangB(n, A0):
    denom = 0
    for i in range(n+1):
        denom += (A0**i)/(factorial(i))
    E1 = ((A0**n)/factorial(n))/denom
    return E1

# def erlangC(n, A0):
#     denom = ((A0**n)/factorial(n))*(n/(n-A0))
#     for i in range(n):
#         denom += (A0**i)/factorial(i)
#     return ((A0**n)/factorial(n))*(n/(n-A0))/denom

def erlangC(n, A0):
    denomSum = 0
    for i in range(n):
        denomSum += (A0**i)/factorial(i)
    denom = A0**n + factorial(n)*(1-A0/n)*denomSum
    return (A0**n)/denom

def meanCallDelay(n, A0, meanCallDuration, probDelay):
    # print(probDelay)
    return probDelay*meanCallDuration/(n-A0)

def meanQueuedDelay(n, A0, meanCallDuration):
    return meanCallDuration/(n-A0)

def erlangC_GOS(n, A0, meanCallDuration, probDelay, maxDelay=0):
    return probDelay*exp(-(n-A0)*maxDelay/meanCallDuration)


In [ ]:
def trafficSimulation(numChannels, numCalls, meanCallDuration, requeue=False, maxDelay=0, holdingDistrib='exp'):

    if holdingDistrib == 'exp':
        callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)
    elif holdingDistrib == 'gamma':
        # callDurations = np.random.gamma(shape=1.2073, scale=meanCallDuration, size=numCalls)
        callDurations = np.random.gamma(shape=1.2073, scale=0.03205, size=numCalls)  # shape from paper, scale from paper: 0.03205 = muCHT*(muPaper/bPaper) = muCHT * (29.6121/35.875)
        # plt.hist(callDurations*60, 50)
        # plt.show()
        # print(callDurations.mean()*60)
    callStarts = np.random.uniform(size=numCalls)
    # if requeue == False:
    #     callStarts = np.random.uniform(size=numCalls)
    # else:
    #     callStarts = np.random.poisson(lam=meanCallDuration, size=numCalls)/2
    #     print(callStarts.mean())
        
    callStarts.sort()
    # plt.plot(callStarts)
    # plt.show()
    # print(callStarts.max())
    # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
    calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

    channelFreeTimes = np.zeros(numChannels)
    
    time = 0
    callsFailed = 0
    callsDelayed = 0
    totalDelay = 0
    for i, call in enumerate(calls):
        time = call[0]
        channelFound = False
        # print("\nTime:", time)
        # print("channelFreeTimes", channelFreeTimes)
        for j in range(numChannels):
            if not(channelFound):
                if channelFreeTimes[j] <= time:
                    # print("Channel {} free. Adding call end time {}".format(j, call[1]))
                    channelFreeTimes[j] = call[1]
                    channelFound = True
                    # print("channelFreeTimes", channelFreeTimes)
        if channelFound == False:
            if requeue == False:
                callsFailed += 1
                # print("Call {} ({}) failed.\n".format(i,call))
            else:
                nextCallFinishTime = channelFreeTimes.min()
                totalDelay += nextCallFinishTime-call[0]
                if totalDelay>maxDelay:
                    callsDelayed += 1
                    # time = nextCallFinishTime # go to next call finish time
                    indices = np.where(channelFreeTimes == nextCallFinishTime)
                    index = indices[0][0]
                    channelFreeTimes[index] = call[1]
                    # print("Call {} delayed, waited until time = {} for channel {},{}".format(i,time,indices,index))
                    # print("Channel {} free. Adding call end time {}".format(index, call[1]))
                    # print("channelFreeTimes", channelFreeTimes)

    if requeue == False:
        # print("False")
        return callsFailed/numCalls, callsFailed
    else:
        return callsDelayed/numCalls, callsDelayed, totalDelay


In [ ]:
meanCallDuration = 2.33/60# 2.33 mins in hrs
n = 57

# minNumCalls = 1
# maxNumCalls = 2000
# numTests = 21
# simulationCallAmounts = np.concatenate((np.linspace(minNumCalls, maxNumCalls, numTests), np.linspace(2500, 6000, 8))).astype(np.int)
simulationCallAmounts = np.linspace(0, 3000, 16).astype(np.int)
# simulationCallAmounts = np.linspace(0, 6000, 31).astype(np.int)
# simulationCallAmounts = np.linspace(1000, 1000, 1).astype(np.int)

repetitionsPerSimulation = 10
print(simulationCallAmounts)

In [ ]:
erlangBs = np.zeros(simulationCallAmounts.shape)
# erlangCs = np.zeros(simulationCallAmounts.shape)
GOS_simulated = np.zeros(simulationCallAmounts.shape)
meanCallDelays = np.zeros(simulationCallAmounts.shape)
meanQueuedDelays = np.zeros(simulationCallAmounts.shape)

print("# Calls:\t\tErlang B:\t\tSimulation:")
for i, numCalls in enumerate(simulationCallAmounts):
    A0 = offeredTraffic(numCalls, meanCallDuration)
    erlangBs[i] = erlangB(n, A0)*100
    # erlangCs[i] = erlangC(n, A0)*100
    # meanCallDelays[i] = meanCallDelay(n, A0, meanCallDuration, erlangCs[i])
    # meanQueuedDelays[i] = meanQueuedDelay(n, A0, meanCallDuration)
    
    totalFailRate, totalFailCount = 0, 0
    for j in range(repetitionsPerSimulation):
        failRate, failsCount = trafficSimulation(n, numCalls, meanCallDuration, holdingDistrib='gamma')
        totalFailRate += failRate
        totalFailCount += failsCount
    meanFailRate = totalFailRate/repetitionsPerSimulation
    meanFailCount = totalFailCount/repetitionsPerSimulation
    GOS_simulated[i] = meanFailRate*100
    # print("Call fail rate in {} simulations = {}% (mean of {} fails in {} calls)\n".format(repetitionsPerSimulation, GOS_simulated[i], meanFailCount, numCalls))


# for i, numCalls in enumerate(simulationCallAmounts):
    print("{}\t\t\t{:.4f}\t\t\t{:.5f}".format(numCalls,erlangBs[i], GOS_simulated[i]))

In [ ]:
# print("Erlang B:\t\t\t Simulation:")
# for i, numCalls in enumerate(simulationCallAmounts):
#     print(GOS_erlang[i], "\t\t", GOS_simulated[i])

plt.plot(offeredTraffic(simulationCallAmounts,meanCallDuration), erlangBs, label='Erlang B')
# plt.plot(simulationCallAmounts, np.clip(erlangCs,0,100), label='Erlang C')
plt.plot(offeredTraffic(simulationCallAmounts,meanCallDuration), GOS_simulated, label='Exponential')
plt.xlabel('Offered traffic (Erlangs)')
plt.ylabel('Grade Of Service (%)')
plt.title("GOS vs number of calls made for n channels = "+str(n))
plt.legend()
plt.show()

# plt.plot(simulationCallAmounts, np.clip(meanCallDelays,0,1)*60, label='meanCallDelays')
# plt.plot(simulationCallAmounts, np.clip(meanQueuedDelays,0,1)*60, label='meanQueuedDelays')
# plt.xlabel('Number of calls')
# plt.ylabel('Mean Delay (mins)')
# plt.legend()
# plt.show()

In [ ]:
erlangBs = np.zeros(simulationCallAmounts.shape)
erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays = np.zeros(simulationCallAmounts.shape)
meanQueuedDelays = np.zeros(simulationCallAmounts.shape)
simProbDelay = np.zeros(simulationCallAmounts.shape)
pMaxDelays = np.zeros(simulationCallAmounts.shape)
maxDelay=0#20/3600
print("# Calls:\tErlang C:\tErlang C w/D\tSimulation:\tCall Delays\tQueued Delays\tSim # Delayed:\tSim Delay Durations:")
for i, numCalls in enumerate(simulationCallAmounts):
    A0 = offeredTraffic(numCalls, meanCallDuration)
    erlangBs[i] = erlangB(n, A0)*100
    erlangCs[i] = erlangC(n, A0)*100
    pMaxDelays[i] = erlangC_GOS(n, A0, meanCallDuration, erlangCs[i], maxDelay=maxDelay)
    meanCallDelays[i] = meanCallDelay(n, A0, meanCallDuration,  np.clip(erlangCs,0,100)[i])
    meanQueuedDelays[i] = meanQueuedDelay(n, A0, meanCallDuration)
    
    totalProbDelay, totalDelayCount, totalDelay = 0, 0, 0
    for j in range(repetitionsPerSimulation):
        probDelay, delayCount, delay = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, maxDelay=maxDelay)
        totalProbDelay += probDelay
        totalDelayCount += delayCount
        totalDelay += delay
    meanProbDelay = totalProbDelay/repetitionsPerSimulation
    meanDelayCount = totalDelayCount/repetitionsPerSimulation
    meanDelay = totalDelay/repetitionsPerSimulation
    simProbDelay[i] = meanProbDelay*100
    # print("Call fail rate in {} simulations = {}% (mean of {} fails in {} calls)\n".format(repetitionsPerSimulation, GOS_simulated[i], meanFailCount, numCalls))


# for i, numCalls in enumerate(simulationCallAmounts):
    print("{}\t\t{:.2f}\t\t{:.2f}\t\t{:.2f}\t\t{:.5f}\t\t{:.5f}\t\t{:.3f}\t\t{:.5f}".format(numCalls, erlangCs[i], np.clip(pMaxDelays,0,100)[i], simProbDelay[i], meanCallDelays[i]*3600, meanQueuedDelays[i]*3600, meanDelayCount, meanDelay*3600))

In [ ]:
plt.plot(simulationCallAmounts, np.clip(erlangCs,0,100), label='Erlang C')
plt.plot(simulationCallAmounts, np.clip(simProbDelay,0,100), label='Exponential with requeue')
plt.plot(simulationCallAmounts, erlangBs, label='Erlang B')
plt.plot(simulationCallAmounts, GOS_simulated, label='Exponential')


plt.xlabel('Number of calls')
plt.ylabel('Grade Of Service (%)')
plt.title("GOS vs number of calls made with max delay = {} seconds".format(str(maxDelay*3600)))
plt.legend()
plt.show()

# plt.plot(simulationCallAmounts, np.clip(meanCallDelays,0,1)*60, label='meanCallDelays')
# plt.plot(simulationCallAmounts, np.clip(meanQueuedDelays,0,1)*60, label='meanQueuedDelays')
# plt.xlabel('Number of calls')
# plt.ylabel('Mean Delay (mins)')
# plt.legend()
# plt.show()